# Ceneo Scraper

##Ekstracja składowych pojedynczej opinii

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review_feature_title-positives ~ div.review-feature_item|pros|
|lista wad|div.review_feature_title-negatives ~ div.review-feature_item|cons|
|dla ilu przydatna|button.vote-yes > span|useful|
|dla ilu nieprzydatna|button.vote-no > span|useless|
|data wystawienia|span.user-post__published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|


# Biblioteki

In [111]:
import os
import json
import requests
from bs4 import BeautifulSoup

## Funkcja do ekstrakcji danych z kodu HTML

In [112]:
def extract(ancestor,selector=None, attribute=None, return_list=False):
    if selector:
        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try: 
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()


## Słownik z strukturą opinii z serwisu Ceneo.pl

In [113]:
selectors = {
            "opinion_id" : (None, "data-entry-id"),       
            "author" : ("span.user-post__author-name",),
            "recommendation" : ("span.user-post__author-recomendation > em",),
            "rating" : ("span.user-post__score-count",),
            "content" : ("div.user-post__text",),
            "pros" : ("div.review_feature_title-positives ~ div.review-feature_item", None, True),
            "cons" : ("div.review_feature_title-negatives ~ div.review-feature_item", None, True),
            "useful" : ("button.vote-yes > span",),
            "useless" : ("button.vote-no > span",),
            "publish_date" : ("span.user-post__published > time:nth-child(1)","datetime"),
            "purchase_date" : ("span.user-post__published > time:nth-child(2)","datetime")
}

## Link do pierwszej strony z opiniami o wskazanym produkcie w serwisie Ceneo.pl


In [114]:
#product_id = "94103927"
product_id = input("podaj kod produktu w serwisie Ceneo.pl:")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

## Ekstrakcja wszystkich opinii o wskazanym produkcie z serwisu Ceneo.pl

In [110]:
all_opinions = []
while (url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text,"html.parser")
    opinions = page_dom.select("div.js_product-review")

    for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *value)
                    for key, value in selectors.items()      
            }
            all_opinions.append(single_opinion)
    try:    
        url = "https://www.ceneo.pl"+extract(page_dom,"a.pagination__next","href")
    except TypeError:
        url = None 


## Zapis pobranych opinii o wskazanym produkcie do pliku JSON

In [108]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")

with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jsonfile:
    json.dump(all_opinions, jsonfile, indent =4, ensure_ascii=False)

In [109]:
print(len(all_opinions))

38
